In [22]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import requests
import os
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
root_directory = os.getcwd()
print(root_directory)

/scratch/zhivar/robust-prototype-learning


In [8]:
def get_embeddings(list_of_ids, list_of_texts, results_embeddings={}):
    for _id, essay in tqdm(
        zip(list_of_ids, list_of_texts), total=len(list_of_ids), leave=False
    ):
        try:
            if str(_id) in results_embeddings.keys():
                continue
            api_key = os.environ.get("OPENAI_API_KEY")
            url = "https://api.openai.com/v1/embeddings"
            headers = {
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json",
            }
            data = {
                "input": essay,
                "model": "text-embedding-ada-002",
                "encoding_format": "float",
            }

            # Send the POST request
            response = requests.post(url, headers=headers, json=data)

            # Check the response status and content
            if response.status_code == 200:
                # Successful request, you can access the response data using response.json()
                response_data = response.json()
                results_embeddings[str(_id)] = response_data
            else:
                # Request failed, print the error message
                print(
                    f"Request failed with status code {response.status_code}: {response.text}"
                )
                continue
        except Exception as e:
            print(e)
            raise e
    return results_embeddings

In [9]:
if os.path.exists(os.path.join(root_directory, "datasets", "embeddings.json")):
    with open(os.path.join(root_directory, "datasets", "embeddings.json"), "r") as f:
        embeddings = json.load(f)
else:
    embeddings = {}

In [ ]:
for dataset in tqdm(["imdb", "ag_news", "dbpedia"], leave=False):
    dataset_directory = os.path.join(root_directory, "datasets", f"{dataset}_dataset")
    for attack_type in tqdm(
        ["deepwordbug", "textfooler", "textbugger", "pwws", "bae"], leave=False
    ):
        for condition in tqdm(["test", "adv"], leave=False):
            condition_file = os.path.join(
                dataset_directory, f"{condition}_{attack_type}.csv"
            )
            condition_df = pd.read_csv(condition_file)
            condition_texts = condition_df["text"].tolist()[:100]

            condition_ids = np.arange(len(condition_texts))

            embeddings[f"{dataset}_{attack_type}_{condition}"] = get_embeddings(
                condition_ids,
                condition_texts,
                embeddings.get(f"{dataset}_{attack_type}_{condition}", {}),
            )

with open(os.path.join(root_directory, "datasets", "embeddings.json"), "w") as f:
    json.dump(embeddings, f, indent=2)

In [28]:
semantic_similarities = dict()
for dataset in tqdm(["imdb", "ag_news", "dbpedia"], leave=False):
    dataset_directory = os.path.join(root_directory, "datasets", f"{dataset}_dataset")
    for attack_type in tqdm(
        ["deepwordbug", "textfooler", "textbugger", "pwws", "bae"], leave=False
    ):
        original_embeddings = embeddings[f"{dataset}_{attack_type}_test"]
        attack_embeddings = embeddings[f"{dataset}_{attack_type}_adv"]

        original_embeddings = {
            key: data["data"][0]["embedding"]
            for key, data in original_embeddings.items()
        }
        attack_embeddings = {
            key: data["data"][0]["embedding"] for key, data in attack_embeddings.items()
        }
        original_keys = set(original_embeddings.keys())
        attack_keys = set(attack_embeddings.keys())
        assert original_keys == attack_keys

        similarities = list()
        for key in tqdm(original_keys):
            original_embedding = original_embeddings[key]
            attack_embedding = attack_embeddings[key]
            similarity = cosine_similarity([original_embedding], [attack_embedding])[0][
                0
            ]
            similarities.append(similarity)
        semantic_similarities[f"{dataset}_{attack_type}"] = (
            np.mean(similarities),
            np.std(similarities),
        )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [29]:
semantic_similarities

{'imdb_deepwordbug': (0.9904409891267784, 0.009918638269768881),
 'imdb_textfooler': (0.9810825357688218, 0.020157756355684467),
 'imdb_textbugger': (0.9837575747487755, 0.019487219057884534),
 'imdb_pwws': (0.983694823582972, 0.01901823487737364),
 'imdb_bae': (0.9941615531904249, 0.006331167633056274),
 'ag_news_deepwordbug': (0.964955895939746, 0.018207566998872746),
 'ag_news_textfooler': (0.9510906159565657, 0.028300085195914332),
 'ag_news_textbugger': (0.973892131308832, 0.017474803796811163),
 'ag_news_pwws': (0.9536064508088522, 0.03144961258670697),
 'ag_news_bae': (0.9846871324982324, 0.019841025832571527),
 'dbpedia_deepwordbug': (0.9622527694389444, 0.025093614152112535),
 'dbpedia_textfooler': (0.9456673008490548, 0.039495608123832765),
 'dbpedia_textbugger': (0.9691112866669972, 0.022947915971711537),
 'dbpedia_pwws': (0.965366771905382, 0.025946964737336816),
 'dbpedia_bae': (0.9721582833109741, 0.05787453080610874)}